## Dependencies

In [57]:
# SOURCE: https://github.com/hhursev/recipe-scrapers
# scraper library dependencies
from recipe_scrapers import scrape_me
import re

# scraping dependencies
import datetime
import requests
import time
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import collections

## Part 1 - saving index links to a list of nested dictionaries

In [58]:
# Saving URL in a variable to pass into the scraper library.
scraper = scrape_me('https://www.simplyrecipes.com/index/')

In [59]:
# returns a list of dictionaries containting all of the <a> tag attributes from the defined website. The attribute names are assigned as dictionary keys and the 
raw_dict = scraper.links()

In [60]:
# for loop that iterates through the dictionary
result = []
for d in raw_dict:
    # Iterates through every key in the dictionary
    for key, value in d.items():
        # If the key is not equal to 'href' and the value doesn't contain ''/recipe' then that pair is skipped.
        if key != "href" or "https://www.simplyrecipes.com/recipes/" not in value:
            # Move onto next iteration
            continue
        txt = ''
        # using part of the URL (key value) to determine the key name
        for i in re.findall("recipes/.*", value):
            txt += i
            title = txt.split('/')[1]
            # appending each key and value to a nested dictionary in a list.

            result.append({title: value})

## Part 2 - code to scrape a recipe's URL.

#### Notes: we can probably implement this code into a for loop and iterate through whatever structure our complete URL list sits within

In [61]:
# boilerplate code to setup Browser functionality during scrape.
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/williamforsyth/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [62]:
# setting URL for requets
resp = requests.get("https://www.simplyrecipes.com/recipes/egg_salad_sandwich/")

# Initiating HTML parser, using BeautifulSoup
soup = BeautifulSoup(resp.text, "html.parser")

# using soup.find to find <div> tags with a class of "recipe-callout", that contains each URL recipe
div_ = soup.find("div", attrs={"class": "recipe-callout"})

# Firstly finding all H2 classes and setting as the key. Then iterating through every list item class within "recipe-callout" and splitting the text as the value for that key
recipes = {"_".join(div_.find("h2").text.split()):
               [x.text for x in div_.findAll("li", attrs={"class": "ingredient"})]}

AttributeError: 'NoneType' object has no attribute 'find'

In [63]:
recipes

{'Quick_and_Easy_Egg_Salad_Sandwich_Recipe': ['1 hard-boiled large egg, peeled and chopped (See Recipe Note)',
  '1-2 tablespoons mayonnaise (to taste)',
  '2 tablespoons chopped celery',
  '1 tablespoon chopped green onion or chives',
  'A pinch of curry powder',
  'Salt and pepper (to taste)',
  'Lettuce',
  '2 slices white, wheat, multigrain, or rye bread, toasted or plain']}

In [64]:
import collections

data = result
dd = collections.defaultdict(list)

for record in data:
    key = list(record)[0]
    value = record[key]
    dd[key].append(value)

In [65]:
dd

defaultdict(list,
            {'diet': ['https://www.simplyrecipes.com/recipes/diet/allergy-friendly/',
              'https://www.simplyrecipes.com/recipes/diet/dairy-free/',
              'https://www.simplyrecipes.com/recipes/diet/gluten-free/',
              'https://www.simplyrecipes.com/recipes/diet/healthy/',
              'https://www.simplyrecipes.com/recipes/diet/low_carb/',
              'https://www.simplyrecipes.com/recipes/diet/paleo/',
              'https://www.simplyrecipes.com/recipes/diet/vegan/',
              'https://www.simplyrecipes.com/recipes/diet/vegetarian/'],
             'main-ingredient': ['https://www.simplyrecipes.com/recipes/main-ingredient/beef/',
              'https://www.simplyrecipes.com/recipes/main-ingredient/cheese/',
              'https://www.simplyrecipes.com/recipes/main-ingredient/chicken/',
              'https://www.simplyrecipes.com/recipes/main-ingredient/egg/',
              'https://www.simplyrecipes.com/recipes/main-ingredient/fish/